## Data augmenation attempt 2 
### EN-DE

reading in all the different augmeneted data sources

In [ ]:
import pandas as pd
from fuzzywuzzy import process

In [ ]:
german_lines = []
english_lines = []
with open("../data-augmented_2/en-de/orginial/rapid2019.de-en.de", encoding="UTF-8") as f:
    for line in f.readlines():
        line = line.replace('\r', '').replace('\n', '').strip()
        if len(line) > 0:
            german_lines.append(line)
with open("../data-augmented_2/en-de/orginial/rapid2019.de-en.en", encoding="UTF-8") as f:
    for line in f.readlines():
        line = line.replace('\r', '').replace('\n', '').strip()
        if len(line) > 0:
            english_lines.append(line)
    

In [ ]:
d = {"orginal":english_lines, "translation":german_lines}
rapid_df = pd.DataFrame(data=d)

In [ ]:
europarl_df =  pd.read_csv('../data-augmented_2/en-de/orginial/europarl-v9.de-en.tsv',sep="\t", error_bad_lines=False, names = ["translation", "orginal"])
europarl_df = europarl_df.dropna()
europarl_df = europarl_df[["orginal", "translation"]]

In [ ]:
newscomm_df =  pd.read_csv('../data-augmented_2/en-de/orginial/news-commentary-v14.tsv',sep="\t", names = ["translation",  "orginal"], error_bad_lines=False)
newscomm_df = newscomm_df.dropna()
newscomm_df = newscomm_df[["orginal", "translation"]]

#### combining all the seperate datframes into one result df

In [ ]:
result_df = pd.concat([rapid_df, europarl_df, newscomm_df], ignore_index=True)

In [ ]:
result_df = result_df.dropna()
result_df["orginal"] = result_df["orginal"].apply(lambda x: x.replace('\r', '').replace('\n', '').strip())
result_df["translation"] = result_df["translation"].apply(lambda x: x.replace('\r', '').replace('\n', '').strip())

In [ ]:
result_df.to_csv("../data-augmented_2/en-de/orginial/result.tsv", index = None, header=True,sep = '\t')

## Sentence similairty 

In [ ]:
en_de_test_df = pd.read_csv("../data/en-de-test/test20.ende.df.short.tsv" ,sep="\t")
#result_df =
pd.read_csv("../data-augmented_2/en-de/orginial/result.tsv", sep = '\t')
en_de_train_df =  pd.read_csv("../data/en-de/train.ende.df.short.tsv",sep="\t")

In [ ]:
import scipy

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
result_df = result_df.dropna()

In [ ]:
corpus=[i for i in newscomm_df["orginal"].to_list() if i != ''and len(i.split(' '))>=3]

In [ ]:
corpus_embeddings = model.encode(corpus)

In [ ]:
queries = en_de_test_df["original"].to_list()
query_embeddings = model.encode(queries)

In [ ]:
len(corpus)

In [ ]:
closest_n = 20
similar_sentence_list = []
score_list = []
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    for idx, distance in results[0:closest_n]:
        similar_sentence_list.append(corpus[idx].strip())
        score_list.append((1-distance))

In [ ]:
d = {"sentence":similar_sentence_list ,"score":score_list}
score_df = pd.DataFrame(data=d)

In [ ]:
score_2000_df = score_df.sort_values(by=['score'], ascending=False).drop_duplicates(subset=["sentence"],keep="first")[0:2000]

In [ ]:
score_2000_df.reset_index(inplace=True,drop=True)
newscomm_df.reset_index(inplace=True,drop=True)

In [ ]:
score_2000_df

In [ ]:
data_aug_df = pd.DataFrame()
for index, sentence in score_2000_df["sentence"].items():
    print(index)
    try:
        wanted_index=None
        wanted_index = newscomm_df[newscomm_df['orginal'] == sentence]
        data_aug_df = pd.concat([data_aug_df,wanted_index], ignore_index=True)
    #soemtimes the text transquest is not exactly the same as the orginial feature of the dev data        
    except IndexError:
        wanted_index = None
        matched_string = process.extractOne(sentence,newscomm_df["orginal"].values.tolist())[0]
        wanted_index = newscomm_df[newscomm_df['orginal'] == matched_string]
        print(sentence)
        print(wanted_index)
        data_aug_df = pd.concat([data_aug_df,wanted_index], ignore_index=True)
        

In [ ]:
data_aug_df = data_aug_df.drop_duplicates(subset=["orginal"],keep="first").reset_index(drop=True)

In [ ]:
data_aug_df.rename(columns={'orginal': 'original'}, inplace=True)
data_aug_df["scores"]= None
data_aug_df["mean"]= None
data_aug_df["z_scores"]= None
data_aug_df["z_mean"]= 2.743416
data_aug_df["model_scores"]= None

In [ ]:
data_aug_df.reset_index(inplace=True)

In [ ]:
train_aug_df = pd.concat([en_de_train_df, data_aug_df], ignore_index=True)

In [ ]:
train_aug_df.to_csv("../data-augmented_2/en-de/augmented/result.tsv",index = None, header=True,sep = '\t')

In [ ]:
train_aug_df